#### **Library Imports**

In [ ]:
### Data Ingestion
import tempfile
from pathlib import Path
import py7zr

# Data Manipulation
import numpy as np
import pandas as pd

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def read_parquet_gzip_7z(filepath: str | Path) -> pd.DataFrame:
    """Read a .parquet.gzip.7z file into a DataFrame.

    Extracts the 7z archive to a temp directory, then reads the inner
    parquet file with pandas. The '.gzip' in the filename refers to
    the parquet-internal compression codec, not external gzip wrapping.
    """
    with tempfile.TemporaryDirectory() as tmpdir:
        with py7zr.SevenZipFile(filepath, mode="r") as archive:
            archive.extractall(path=tmpdir)
        extracted = list(Path(tmpdir).iterdir())
        if not extracted:
            raise ValueError(f"No files found inside {filepath}")
        return pd.read_parquet(extracted[0])

In [ ]:
path = ('../data/5FT0192.7z')
df = read_parquet_gzip_7z(path)

In [ ]:
df.head()

In [ ]:
df.columns.tolist()

In [ ]:
# Throttle Override Status - 6: Override Active, Else: iQC Active
df['iQC1.iQCMode'].value_counts().sort_index()

In [ ]:
# Brake Switch Status - 0: Not Pressed, 1: Pressed
df['CCVS1_Cab_Controller.BrakeSwitch'].value_counts()

In [ ]:
# Brake Switch Status - 0: Not Pressed, 1: Pressed
df['CCVS1_Engine.BrakeSwitch'].value_counts()

#### Override Events
1. Throttle override
2. Brake override
3. Cruise disengage override
4. Speed governer / iQC override
5. PCC system override

In [ ]:
columns = ['override_id', 'override_timestamp', 'override_type', 'cruise_active_pre', 'vehicle_speed', 'driver_input_type', 'system_override_flag']
overrides = pd.DataFrame(columns=columns)